# LCA of Germany's grid status quo

In [1]:
import bw2data as bd
import bw2calc as bc

In [3]:
bd.projects.set_current("bw25_ei310_premise2.3.0dev1_new")

In addition to the cumulated LCIA results, we want to investigate the contributions of certain grid components, materials and of the underlying directly emitting processes.

## Contributions of components and materials


Setup:

In [4]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

method = ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')

Now we calculate the LCI the standard way, but factorize the matrices because we will do additional calculations with the same matrices in the next step.

In [5]:
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)

/Users/timodiepers/anaconda3/envs/paper-plca-grid-expansion/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


Calculating the LCIA results for the component and material levels:

In [6]:
component_scores = {}
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

Saving the results in a DataFrame and cleaning up the labels:

In [7]:
import pandas as pd

def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminum",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df = pd.DataFrame(component_scores)

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials


,cables,overhead lines,substations,switchgears,transformers
SF6,0.000000e+00,0.000000e+00,0.000000e+00,5.104339e+08,0.000000e+00
aluminum,1.611810e+10,1.813097e+10,0.000000e+00,3.430312e+08,8.409121e+08
concrete,0.000000e+00,2.644550e+09,1.984782e+08,0.000000e+00,0.000000e+00
copper,2.688939e+09,1.372149e+09,0.000000e+00,4.416130e+07,4.872204e+08
iron & steel,4.986620e+08,1.196198e+10,9.920280e+08,4.293074e+07,1.790981e+09
other materials,3.887110e+08,5.803416e+08,0.000000e+00,3.630961e+07,5.564668e+08
plastics,4.524003e+09,7.422802e+06,0.000000e+00,0.000000e+00,0.000000e+00


## Process contributions

To get the information which processes are the direct emitters, but keep the connection to the responsible materials, we use the initially created "aggregated material" activities that essentially sum up the demands for a certain material across all grid components.


First, we collect the aggregated materials nodes:

In [8]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

Now, we do the individual LCAs, and aggregate the results by the reference product of the underlying directly emitting processes:

In [9]:
import bw2analyzer as ba

lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=5000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

/Users/timodiepers/anaconda3/envs/paper-plca-grid-expansion/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


We save the results in a DataFrame again and clean up the names:

In [10]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminum (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminum,concrete,copper,iron & steel,other materials,plastics
SF6,1.422645e+06,1.267405e+04,2.223569e+05,6.140949e+04,4.897193e+08,5.577546e+04
aluminum (process emissions),5.105790e+09,5.366027e+06,2.579304e+07,6.671856e+06,3.932040e+06,7.377375e+06
clinker,7.863092e+07,1.896096e+09,1.536054e+08,2.286787e+08,2.659729e+07,2.783885e+07
coal,2.957688e+09,9.100639e+07,2.029298e+08,2.571239e+09,5.929527e+07,2.111420e+08
electricity,1.736756e+10,2.965919e+08,2.173636e+09,1.755735e+09,5.310811e+08,1.193198e+09
heat,5.998438e+09,5.897245e+07,4.441826e+08,4.083278e+08,2.662038e+08,1.620390e+08
iron & steel (process emissions),2.154794e+08,2.593795e+07,8.500712e+07,7.932731e+09,3.179397e+07,6.869344e+07
other processes,9.178210e+08,1.314109e+08,8.249137e+08,8.361464e+08,4.121090e+08,2.605713e+09
transport,2.790183e+09,3.317716e+08,6.821791e+08,1.546995e+09,2.573933e+08,2.553692e+08


## Prepare sankey diagram

Now we have all the information we need, we just need to get the data in the right format to feed our Sankey Diagram.


In [11]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Grid to components:


In [12]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [13]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [14]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a DataFrame and save as .csv:

In [15]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])
df_sankey.value = df_sankey.value / 1e9
df_sankey.columns = ["source", "target", "value"]
df_sankey.to_csv("data/sankey_data.csv", index=False)

In [16]:
df_sankey

,source,target,value
0,cables,grid status quo,24.218414
1,overhead lines,grid status quo,34.697415
2,substations,grid status quo,1.190506
3,switchgears,grid status quo,0.976867
4,transformers,grid status quo,3.675580
...,...,...,...
76,electricity,plastics,1.193198
77,heat,plastics,0.162039
78,iron & steel (process emissions),plastics,0.068693
79,other processes,plastics,2.605713


The final figure for the paper was created using the Javascript library D3Blocks. The code is available on Observable: https://observablehq.com/@timo-diepers-ws/sankey-grid-status-quo-de. 